# Get Pinnacle Odds

In [30]:
from bs4 import BeautifulSoup
import requests
import datetime #as dt
from datetime import date 
from datetime import datetime
import pandas as pd 
import re 
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
import sys
import warnings
from datetime import timedelta
import time as timp 
import dateutil.parser
import mysql.connector
import sqlalchemy
from mysql.connector import Error

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
#Get df of leagues 

def get_pinn_leagues(URL):
    chrome_path= r'C:\Users\Iris\Documents\Nico\NicoUni\Practicum\Python\Projects\WebDrivers\chromedriver.exe'
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(
        executable_path=chrome_path, options=chrome_options)
    browser.get(URL)
    #browser.implicitly_wait(20)

    timp.sleep(20)

    htmlSource = browser.page_source
    #browser.implicitly_wait(20)

    soup= BeautifulSoup(htmlSource, 'lxml')
    browser.quit()
    league_name= []
    links= []
    for element in soup.select('a[class*="style_supportFavoritesList"]'):
        links.append('https://www.pinnacle.com'+element['href'])
        league_name.append(element.text)
    
    link_frame= pd.DataFrame({
        'league': league_name,
        'link': links})
    
    p= re.compile('.*[^0-9]')
        
    for i in np.arange(0,len(link_frame)):
        link_frame['league'][i]= p.search(link_frame['league'][i]).group()
        
    return(link_frame)


In [31]:
def get_pinn_odds(URL,which):
    #import time as timp
    chrome_path= r'C:\Users\Iris\Documents\Nico\NicoUni\Practicum\Python\Projects\WebDrivers\chromedriver.exe'
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
    #browser.implicitly_wait(10)
    browser.get(URL)
    timp.sleep(5)
    #browser.find_element_by_tag_name("button")
    soup= BeautifulSoup(browser.page_source, 'lxml')
    browser.quit()
    df= pd.DataFrame()
    error= []

    df= pd.DataFrame()
    
    for element in soup.find_all('div',{'class':['style_dateBar__3gX3j','style_dateBar__2PS4O']}):
        count= len(element.text)
        if 'Today' in element.text:
            date= pd.to_datetime(datetime.now(), format="%d/%m/%Y")
        if 'Tomorrow' in element.text:
            days_add= timedelta(days= 1)
            date= (pd.to_datetime(datetime.now(),format="%d/%m/%Y")+days_add)
        else:
            if which== 'yes':
                try:
                    temp_date= dateutil.parser.parse(element.text.split(',',1)[1][1:].replace(',',''))
                    date= pd.to_datetime(temp_date, format= '%d/%m/%Y')
                except:
                    date= element.text
            else:
                 pass
        for tag in element.next_siblings:
            if tag['class'] == ['style_dateBar__3gX3j'] or tag['class']==['style_dateBar__2PS4O']:
                break
            if tag['class']== ['style_row__2ww2Y'] or tag['class']== ['style_row__2ww2Y', 'style_lastRow__3Y2le']\
            or tag['class']== ['style_row__3_aBC'] or tag['class']== ['style_row__3_aBC', 'style_lastRow__3h8Pm']:
                try:
                    teams= tag.find_all('span',{'class':["style_participantName__2tjoj ellipsis", "style_participantName__CNiJz ellipsis"]})
                    play_time= tag.find('span',{'class':["style_time__2GvnE ellipsis","style_time__1_zpO ellipsis"]})
                    odds= tag.find_all('span',{'class':'price'})
                    fest= pd.DataFrame({'date':date,
                                    'scrape_time':pd.to_datetime(datetime.now(), format="%d/%m/%Y %H:%M"),
                                    'time':play_time.text,
                         'home_team': teams[0].text,
                         'away_team': teams[1].text,
                         'PinnacleHome': odds[0].text,
                         'PinnacleDraw': odds[1].text,
                         'PinnacleAway': odds[2].text}, index= pd.Series(0))
                    
                    df= df.append(fest)                        
                except:
                    continue
    return(df.reset_index(drop= True))

In [32]:
# Very Important script
#links= get_pinn_leagues('https://www.pinnacle.com/en/soccer/leagues')
links= pd.read_csv('links.csv')
results= pd.DataFrame()
errors= []

for i,element in enumerate(links['link']):
    try:
        df= get_pinn_odds(element,'yes')
        df['league']= links['league'][i]
        results= results.append(df)       
    except:
        errors.append(element) 

In [33]:
results.loc[results['time']== '24:00','time']= '00:00'
results.loc[results['time']== '24:15','time']= '00:15'
results.loc[results['time']== '24:30','time']= '00:30'
results.loc[results['time']== '24:45','time']= '00:45'


def change_date(x):
    if x == 'Today':
        return(datetime.strftime(datetime.now(), format="%Y-%m-%d"))
    else:
        return(datetime.strftime(x, '%Y-%m-%d'))   


results.scrape_time = results.scrape_time.apply(lambda x: x.strftime(format= "%Y-%m-%d %H:%M"))
results.scrape_time= pd.to_datetime(results.scrape_time)
results.date= results.date.map(change_date)
results.date= pd.to_datetime(results.date.astype(str)+' '+results.time.astype(str))
results['difference_to_scrape']= results.date-results.scrape_time
results.difference_to_scrape= results.difference_to_scrape.dt.total_seconds()/3600
results= results.drop('time',axis=1)
results[['PinnacleHome','PinnacleDraw','PinnacleAway']]= results[['PinnacleHome','PinnacleDraw','PinnacleAway']].astype('float')

probs_with_margin= (1/results['PinnacleHome']+1/results['PinnacleDraw']+1/results['PinnacleAway'])

results['ProbsPinnacleHome']= 1/results['PinnacleHome']/probs_with_margin
results['ProbsPinnacleDraw']= 1/results['PinnacleDraw']/probs_with_margin
results['ProbsPinnacleAway']= 1/results['PinnacleAway']/probs_with_margin

# Get Oddschecker Data

In [34]:
def scrape_odds(link):
    try:
        URL= link
        hdr = {'User-Agent': 'Mozilla/5.0'}
        page= requests.get(URL, headers= hdr)
        maps= page.text 
        soup= BeautifulSoup(maps, 'lxml')
        
# Bookmaker Names
        names= []
        for element in soup.find_all('a',{"class": "bk-logo-main-90 bk-logo-click"}):
            names.append(element['title'])
    
        names.insert(0, 'Outcome')

# The odds and Team Names 
        odds= []
        teams= []

        soup_odds= soup.find('tbody')

        for row in soup_odds.find_all('tr'):
            cells= row.find_all('td')
            outcome= cells[0].find('span').find('span')['data-name']
            opus= [x['data-odig'] for x in cells if x.has_attr('data-odig')]
            opus.insert(0, outcome)
            odds.append(opus)
            teams.append(outcome)

        date= soup.find('div',{'class':'event'}).text
        fixture= soup.find('h1').text
        scrape_time= pd.to_datetime(datetime.now(), format="%d/%m/%Y %H:%M:%S")

        data= pd.DataFrame(odds, columns= names)

        p= re.compile('^.+?(?=\ v)')

        home_team= p.search(fixture).group(0)#[:-1]

        lop= list(data['Outcome'])

        if home_team== lop[0] and lop[1] != 'Draw':
            order= [1,2,0]
            lop= [lop[i] for i in order]

        if home_team== lop[0] and lop[1] == 'Draw':
            order= [2,1,0]
            lop= [lop[i] for i in order]
            
        if home_team== lop[1] and lop[2] == 'Draw':
            order= [0,2,1]
            lop= [lop[i] for i in order]
            
        if home_team== lop[1] and lop[2] != 'Draw':
            order= [2,0,1]
            lop= [lop[i] for i in order]
            
        if home_team== lop[2] and lop[1] == 'Draw':
            order= [0,1,2]
            lop= [lop[i] for i in order]
            
        if home_team== lop[2] and lop[1] != 'Draw':
            order= [1,0,2]
            lop= [lop[i] for i in order]
    
        data['Outcome']= pd.Categorical(data['Outcome'], lop)
        data= data.sort_values('Outcome').reset_index(drop= True)

        a_list= list(data.columns[1:])
        b_list= ['H','D','A']

        oo= ['{}{}'.format(a,b) for b in b_list for a in a_list]

        oo.sort()

        for i in np.arange(0,len(oo), 3):
            home= oo[i+2]
            away= oo[i]
            oo[i]= home
            oo[i+2]= away
 
        oo.insert(0, 'scrape_time')
        oo.insert(0, 'date')
        oo.insert(0, 'away_team')
        oo.insert(0,'home_team')

        new= pd.DataFrame(columns= oo, index= np.arange(0,1))

        new['home_team']= data['Outcome'][2]
        new['away_team']= data['Outcome'][0]

        date= soup.find('div',{'class':'event'}).text.split(',',1)[1][1:].\
        lower().replace("rd", "").replace("nd", "").replace("st", "").replace("th", "")

        num= [i for i,x in enumerate(date) if x.isspace()][1]

        date= date[:num]+ ' '+ str(scrape_time.year) + date[num:]

        date= pd.to_datetime(datetime.strptime(date, '%d %B %Y %H:%M'),format= "%Y-%m-d %H:%M")
        hours_added = timedelta(hours = 1)
        new['date']= date+hours_added
        new['scrape_time']= scrape_time

        for name, stuff in data.drop('Outcome',axis=1).iteritems():
            for snam, snugg in new.iteritems():
                if name in snam and snam.endswith('A'):
                    snugg[0]= stuff[0]
                if name in snam and snam.endswith('D'):
                    snugg[0]= stuff[1]         
                if name in snam and snam.endswith('H'):
                    snugg[0]= stuff[2]
            
        new[new.columns.difference(['home_team', 'away_team','date','scrape_time'])]= new[new.columns.difference(['home_team', 'away_team','date','scrape_time'])].astype('float')
        return(new)
    
    except:
        new= pd.DataFrame()
        return(new)

In [35]:
def get_leagues(link):
    lan= open('länder.txt','r').readlines()
    land= [x[:-1] for x in lan]
    URL= link
    hdr = {'User-Agent': 'Mozilla/5.0'}
    page= requests.get(URL, headers= hdr)
    maps= page.text 
    soup= BeautifulSoup(maps, 'lxml')
    links= []
    for element in soup.find_all('loc'):
        s= element.text
        if s.count('/') < 6 and any(thing in s for thing in land) and not s.endswith('winner'):
            links.append(s)
    return(links)

In [36]:
errors= []
oddschecker= pd.DataFrame()
links= get_leagues('https://www.oddschecker.com/sport/football/sitemap.xml')
for thing in links:
    hdr = {'User-Agent': 'Mozilla/5.0'}
    page= requests.get(thing, headers= hdr)
    maps= page.text 
    soup= BeautifulSoup(maps, 'lxml')
    if len(soup.findAll('td',{'class':"bh-date beta-caption2"})) > 2:  
        splitter= str(soup.findAll('td',{'class':"bh-date beta-caption2"})[2])
        soup= BeautifulSoup(str(soup).split(splitter)[0])    
    league= thing[thing.rindex('/')+1:]
    for element in soup.find_all('a',{'class':'beta-callout full-height-link whole-row-link'}):
        try:
            new= scrape_odds('https://www.oddschecker.com'+element['href'])
            new['league']= league
            oddschecker= oddschecker.append(new)
        except:
            errors.append(element) 
            
oddschecker= oddschecker.reset_index(drop= True)

# Merge the Data

In [37]:
#Make column names the same 
oddschecker= oddschecker.sort_values('date').reset_index(drop= True)
results= results.sort_values('date').reset_index(drop= True)

for i in range(0, len(oddschecker)):
    for j in range (0,len(results)):
        if (oddschecker['date'][i]== results['date'][j]):
            if (oddschecker['home_team'][i] in results['home_team'][j])\
            | (results['home_team'][j] in oddschecker['home_team'][i])\
            | (results['away_team'][j] in oddschecker['away_team'][i])\
            | (oddschecker['away_team'][i] in results['away_team'][j]):
                results['home_team'][j]= oddschecker['home_team'][i]
                results['away_team'][j]= oddschecker['away_team'][i]
                
odds_merged= oddschecker.merge(results,on= ['date','home_team','away_team'])

In [38]:
odds_merged= odds_merged.drop_duplicates(subset= ['home_team','away_team']).reset_index(drop= True)

In [39]:
bookies= ['10BetH', '10BetD',
       '10BetA', '888sportH', '888sportD', '888sportA', 'Bet VictorH',
       'Bet VictorD', 'Bet VictorA', 'Bet365H', 'Bet365D', 'Bet365A',
       'Betfair SportsbookH', 'Betfair SportsbookD', 'Betfair SportsbookA',
       'BetfairH', 'BetfairD', 'BetfairA', 'BetfredH', 'BetfredD', 'BetfredA',
       'BetwayH', 'BetwayD', 'BetwayA', 'Boyle SportsH', 'Boyle SportsD',
       'Boyle SportsA', 'GentingBetH', 'GentingBetD', 'GentingBetA',
       'Mansion BetH', 'Mansion BetD', 'Mansion BetA', 'MatchbookH',
       'MatchbookD', 'MatchbookA', 'NovibetH', 'NovibetD', 'NovibetA',
       'Paddy PowerH', 'Paddy PowerD', 'Paddy PowerA', 'RedzoneH', 'RedzoneD',
       'RedzoneA', 'SkybetH', 'SkybetD', 'SkybetA', 'Smarkets SportsbookH',
       'Smarkets SportsbookD', 'Smarkets SportsbookA', 'SmarketsH',
       'SmarketsD', 'SmarketsA', 'Sport NationH', 'Sport NationD',
       'Sport NationA', 'Sporting IndexH', 'Sporting IndexD',
       'Sporting IndexA', 'SpreadexH', 'SpreadexD', 'SpreadexA', 'UnibetH',
       'UnibetD', 'UnibetA', 'VBetH', 'VBetD', 'VBetA', 'William HillH',
       'William HillD', 'William HillA']

exchanges2= ['Betfair SportsbookH','Betfair SportsbookD','Betfair SportsbookA','BetfairH','BetfairD','BetfairA',\
            'MatchbookH','MatchbookD','MatchbookA','SmarketsH','SmarketsD','SmarketsA','Smarkets SportsbookH','Smarkets SportsbookD','Smarkets SportsbookA']


exchanges= ['BetfairH','BetfairD','BetfairA',\
            'MatchbookH','MatchbookD','MatchbookA','SmarketsH','SmarketsD','SmarketsA']
homes= [x for x in bookies if x.endswith('H')]
draws= [x for x in bookies if x.endswith('D')]
away= [x for x in bookies if x.endswith('A')]

homes2= [x for x in bookies if x.endswith('H') and x not in exchanges]
draws2= [x for x in bookies if x.endswith('D') and x not in exchanges]
away2= [x for x in bookies if x.endswith('A') and x not in exchanges ]

homes3= [x for x in bookies if x.endswith('H') and x in exchanges]
draws3= [x for x in bookies if x.endswith('D') and x in exchanges]
away3= [x for x in bookies if x.endswith('A') and x in exchanges ]

homes4= [x for x in bookies if x.endswith('H') and x not in exchanges2]
draws4= [x for x in bookies if x.endswith('D') and x not in exchanges2]
away4= [x for x in bookies if x.endswith('A') and x not in exchanges2]

In [40]:
odds_merged['time_lag']= (odds_merged.scrape_time_x - odds_merged.scrape_time_y).dt.total_seconds()
odds_merged['max_oddsH']= odds_merged[homes].max(axis= 1)
odds_merged['max_oddsD']= odds_merged[draws].max(axis= 1)
odds_merged['max_oddsA']= odds_merged[away].max(axis= 1)

odds_merged['max_oddsH2']= odds_merged[homes2].max(axis= 1)
odds_merged['max_oddsD2']= odds_merged[draws2].max(axis= 1)
odds_merged['max_oddsA2']= odds_merged[away2].max(axis= 1)

odds_merged['max_oddsH3']= odds_merged[homes3].max(axis= 1)
odds_merged['max_oddsD3']= odds_merged[draws3].max(axis= 1)
odds_merged['max_oddsA3']= odds_merged[away3].max(axis= 1)

odds_merged['max_oddsH4']= odds_merged[homes4].max(axis= 1)
odds_merged['max_oddsD4']= odds_merged[draws4].max(axis= 1)
odds_merged['max_oddsA4']= odds_merged[away4].max(axis= 1)

odds_merged['max_BookieH']= odds_merged[homes].idxmax(axis=1)
odds_merged['max_BookieD']= odds_merged[draws].idxmax(axis= 1)
odds_merged['max_BookieA']= odds_merged[away].idxmax(axis= 1)

odds_merged['max_BookieH2']= odds_merged[homes2].idxmax(axis=1)
odds_merged['max_BookieD2']= odds_merged[draws2].idxmax(axis= 1)
odds_merged['max_BookieA2']= odds_merged[away2].idxmax(axis= 1)

odds_merged['max_BookieH3']= odds_merged[homes3].idxmax(axis=1)
odds_merged['max_BookieD3']= odds_merged[draws3].idxmax(axis= 1)
odds_merged['max_BookieA3']= odds_merged[away3].idxmax(axis= 1)

odds_merged['max_BookieH4']= odds_merged[homes4].idxmax(axis=1)
odds_merged['max_BookieD4']= odds_merged[draws4].idxmax(axis= 1)
odds_merged['max_BookieA4']= odds_merged[away4].idxmax(axis= 1)

odds_merged['marginH']= odds_merged['ProbsPinnacleHome']*(odds_merged['max_oddsH']-1)-(1-odds_merged['ProbsPinnacleHome'])
odds_merged['marginD']= odds_merged['ProbsPinnacleDraw']*(odds_merged['max_oddsD']-1)-(1-odds_merged['ProbsPinnacleDraw'])
odds_merged['marginA']= odds_merged['ProbsPinnacleAway']*(odds_merged['max_oddsA']-1)-(1-odds_merged['ProbsPinnacleAway'])

odds_merged['marginH2']= odds_merged['ProbsPinnacleHome']*(odds_merged['max_oddsH2']-1)-(1-odds_merged['ProbsPinnacleHome'])
odds_merged['marginD2']= odds_merged['ProbsPinnacleDraw']*(odds_merged['max_oddsD2']-1)-(1-odds_merged['ProbsPinnacleDraw'])
odds_merged['marginA2']= odds_merged['ProbsPinnacleAway']*(odds_merged['max_oddsA2']-1)-(1-odds_merged['ProbsPinnacleAway'])

odds_merged['marginH3']= odds_merged['ProbsPinnacleHome']*(odds_merged['max_oddsH3']-1)-(1-odds_merged['ProbsPinnacleHome'])
odds_merged['marginD3']= odds_merged['ProbsPinnacleDraw']*(odds_merged['max_oddsD3']-1)-(1-odds_merged['ProbsPinnacleDraw'])
odds_merged['marginA3']= odds_merged['ProbsPinnacleAway']*(odds_merged['max_oddsA3']-1)-(1-odds_merged['ProbsPinnacleAway'])

odds_merged['marginH4']= odds_merged['ProbsPinnacleHome']*(odds_merged['max_oddsH4']-1)-(1-odds_merged['ProbsPinnacleHome'])
odds_merged['marginD4']= odds_merged['ProbsPinnacleDraw']*(odds_merged['max_oddsD4']-1)-(1-odds_merged['ProbsPinnacleDraw'])
odds_merged['marginA4']= odds_merged['ProbsPinnacleAway']*(odds_merged['max_oddsA4']-1)-(1-odds_merged['ProbsPinnacleAway'])

# Save Data to SQL-Server (Backup CSV Script)

In [41]:
#floats= odds_merged.select_dtypes('float').columns
#datetimes= odds_merged.select_dtypes('datetime').columns

#odds_merged[datetimes]= odds_merged[datetimes].astype('str')
#odds_merged[floats]= odds_merged[floats].astype('float16')

#old= pd.read_csv('odds_merged.csv')

#new= old.append(odds_merged)
#new.to_csv('odds_merged.csv',index= False)

In [42]:
#odds_merged['date']= pd.to_datetime(odds_merged['date'])
#odds_merged['scrape_time_x']= pd.to_datetime(odds_merged['scrape_time_x'])
#odds_merged['scrape_time_y']= pd.to_datetime(odds_merged['scrape_time_y'])
cols= [x.replace(' ','') for x in list(odds_merged.columns)]
odds_merged.columns= cols
odds_columns= list(odds_merged.columns)[0:76]
col_test1= ['home_team','away_team','date','scrape_time_x','max_oddsH','max_oddsD','max_oddsA','max_BookieH','max_BookieD','max_BookieA','marginH','marginD','marginA']
col_test2= ['home_team','away_team','date','scrape_time_x','max_oddsH2','max_oddsD2','max_oddsA2','max_BookieH2','max_BookieD2','max_BookieA2','marginH2','marginD2','marginA2']
col_test3= ['home_team','away_team','date','scrape_time_x','max_oddsH3','max_oddsD3','max_oddsA3','max_BookieH3','max_BookieD3','max_BookieA3','marginH3','marginD3','marginA3']
col_test4= ['home_team','away_team','date','scrape_time_x','max_oddsH4','max_oddsD4','max_oddsA4','max_BookieH4','max_BookieD4','max_BookieA4','marginH4','marginD4','marginA4']
probls= ['home_team','away_team','date','scrape_time_x','ProbsPinnacleHome','ProbsPinnacleDraw','ProbsPinnacleAway']
fixtures= odds_merged[['home_team','away_team','date','scrape_time_x','scrape_time_y','time_lag','difference_to_scrape','league_x','league_y']]
odds= odds_merged[odds_columns]
test_group1= odds_merged[col_test1]
test_group2= odds_merged[col_test2]
test_group3= odds_merged[col_test3]
test_group4= odds_merged[col_test4]
probbs= odds_merged[probls]

In [43]:
db_username= "root"
db_password= '4kS8GdBm!'
db_ip= "localhost"
db_name= 'betting'

db_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(db_username, db_password, 
                                                      db_ip, db_name))

fixtures.to_sql('fixtures',db_connection,if_exists= 'append', index= False)
odds.to_sql('odds',db_connection,if_exists= 'append', index= False)
test_group1.to_sql('test_group1',db_connection,if_exists= 'append', index= False)
test_group2.to_sql('test_group2',db_connection,if_exists= 'append', index= False)
test_group3.to_sql('test_group3',db_connection,if_exists= 'append', index= False)
test_group4.to_sql('test_group4',db_connection,if_exists= 'append', index= False)
probbs.to_sql('probabilities',db_connection,if_exists= 'append', index= False)

In [44]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")
        


def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")
        
def get_colnames(database, tablename):
    connection = create_db_connection("localhost", "root", '4kS8GdBm!', database)
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute("""SELECT * FROM {table}""".format(table= tablename))
        result= [i[0] for i in cursor.description]
        return result
    except Error as err:
        print(f"Error: '{err}'")


In [181]:
#group2.query('date <= @dated')
#dated= pd.to_datetime(datetime.now(), format="%d/%m/%Y").date()+timedelta(days=2)
#connection = create_db_connection("localhost", "root", '4kS8GdBm!', 'betting')
#ll= read_query(connection, """SELECT * FROM test_group4""")
#group2= pd.DataFrame(ll, columns= get_colnames('betting','test_group4'))
#today= group2[(group2['date'].dt.date == dated)].reset_index(drop= True)
#today= today.loc[today.groupby(['home_team','away_team'])['scrape_time_x'].idxmax()].reset_index(drop= True)

#today.groupby(['home_team','away_team'])['scrape_time_x'].idxmin()

In [180]:
#today.head(20)
#(today[['marginH2','marginD2','marginA2']]>0)
#positives= today.query('(marginH4 >0 & max_oddsH4< 6)| (marginD4>0 & max_oddsD4< 6) | (marginA4>0 & max_oddsA4< 6)')
#positives.sort_values('date')

In [182]:
#len(spi.league.unique())

#quoten= pd.DataFrame(read_query(connection, """SELECT * FROM test_group4;"""), columns= get_colnames('betting','test_group4'))

1. **Fix the link fetcher**
2. Get results!! 
3. **Make an analysis of lost data**
4. **Why has France so many postive values games**? --> Rerun the analysis with the unique games